In [1]:
import pandas as pd

# 데이터 경로

data_path = './porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv' , index_col = 'id')
test = pd.read_csv(data_path + 'test.csv' , index_col = 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv' , index_col ='id')


In [2]:
all_data = pd.concat([train , test] , ignore_index= True)
all_data = all_data.drop('target' , axis = 1) # 타깃값 제거

all_data

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2,5,1,0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1,7,0,0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4,9,1,0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2,0,1,0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,0,1,6,0,0,0,1,0,0,0,...,4,2,3,4,0,1,0,0,1,0
1488024,5,3,5,1,0,0,0,1,0,0,...,6,2,2,11,0,0,1,1,0,0
1488025,0,1,5,0,0,1,0,0,0,0,...,5,2,2,11,0,1,1,0,0,0
1488026,6,1,5,1,0,0,0,0,1,0,...,1,1,2,7,1,1,0,0,0,0


In [3]:
all_features = all_data.columns # 전체 피처
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

## 명목형 피처 원-핫 인코딩

In [4]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]
# 이름에 cat이 포함된 피처가 명목형 피처이다.
onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성

# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [5]:
# 추가로 제거할 피처
drop_features = ['ps_ind14' , 'ps_ind_10_bin' , 'ps_ind_11_bin' , 'ps_ind_12_bin' , 'ps_ind_13_bin' , 'ps_car_14' ]

# 1> 명목형 피처 , 2> calc 분류의 피처 , 3> 추가 제거할 피처를 제외한 피처

remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and
                          'calc' not in feature and
                          feature not in drop_features)]
# cat(명목형) 피처 , calc(피처) , 추가 제거할 6개피처를 제외한 나머지 피처를 remaining_features 에 저장

In [7]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]) , encoded_cat_matrix] , format= 'csr')

all_data_sprs

# CSR  형식으로 바꾸어 hstack()으로 행렬을 수평 방향으로 합친다.

<1488028x202 sparse matrix of type '<class 'numpy.float64'>'
	with 37644877 stored elements in Compressed Sparse Row format>

In [8]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train :]

y = train['target'].values

## 지니계수

In [9]:
# 지니계수란?

# 소득 불평등 정도를 나타내는 지표. 지니계수가 작을수록 소득 수준이 평등하고, 클수록 불평등함을 의미한다.

In [10]:
# 정규화 지니계수 계산 함수

# 정규화란 값의 범위를 0~1 사이로 조정한다는 의미, 정규화 지니계수는 값이 0에 가까울수록 성능이 나쁘고, 1에 가까울 수록 성능이 좋다는 의미가 된다.

In [11]:
import numpy as np

def eval_gini(y_true , y_pred):
    # 실제값과 예측값의 크기가 서로 같은지 확인(값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1/ n_samples ,1 , n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수

    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선

    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계쑤

    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [13]:
# LightGBM 용 gini() 함수

def gini(preds , dtrain):
    labels = dtrain.get_label()

    return 'gini' , eval_gini(labels , preds ) , True

# 'gini' : 평가지표이름 , eval_gini(labels,preds) : 평가점수 , True : 평가 점수가 높을수록 좋은지 여부

## 모델 훈련 및 성능 검증

In [14]:
# OOF(Out of Fold prediction) 예측 방식

# K 폴드 교차 검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식이다.

# K 폴드 교차 검ㅈ응을 하면서 폴드마다 1> 훈련 데이터로 모델을 훈련하고, 2> 검증 데이터로 모델 성능을 측정하며 , 3> 테스트 데이터로 최종 타깃 확률도 예측한다. 훈련된 모델로 마지막에 한 번만 예측하는 것이 아니다. 각 폴드별 모델로 여러번 예측해 평균을 내는 방식이다.

In [15]:
# OOF 방식으로 LightGBM 훈련
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기

folds = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1991)

# 층화 K 폴드 교차 검증기는 타깃값이 불균형하므로 K폴드가 아닌 층화 K폴드를 수행하는 게 바람직하다. 층화 K폴드는 타깃값이 균등하게
# 폴드를 나누는 방식이기 때문이다.


# n_splits 파라미터로 전달한 수만큼 폴드를 나눈다. 여기서는 5개로 나누었다. shuffle = True 를 전달하면 폴드를 나눌때 데이터를 섞어준다.

In [16]:
# LightGBM의 하이퍼파라미터를 설정한다. LightGBM은 하이퍼파라미터를 갖고 있지만, 여기서는 4가지만 설정한다.

params = {'objective' : 'binary' , 'learning_rate' : 0.01 , 'force_row_wise' : True , 'random_state' : 0}

# 이진분류 문제이므로 objective 파라미터는 binary로 설정했다. 학습률은 0.01로, 랜덤 스테이트 값은 9으로 설정했다.
# force_row_wise : True 는 경고 문구를 없애려고 추가한 파라미터이다.

In [18]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])

# ==> oof_val_preds 는 검증 데이터를 활용해 예측한 확률값을 저장하는 배열이다. K 폴드로 나누어도 훈련 데이터 전체가 결국엔 한 번씩 검증 데이터로 활용된다. 따라서 oof_val_preds 배열 크기는 훈련 데이터와 같아야 한다.
# 훈련 데이터 개수는 X.shpae[0]으로 구한다.

# OOF 방식으로 훈련된 모델로 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

# oof_test_preds는 테스트 데이터를 활용해 예측한 확률값을 저장하는 배열이다. 최종 제출에 사용할 값이므로 크기는 테스트 데이터와 같아야한다. 테스트 데이터 개수는 X_test.shape[0]으로 구한다.

In [21]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련 , 검증 , 예측

for idx, (train_idx , valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40 , f'폴드 {idx +1} / 폴드 {folds.n_splits}' , '#'*40)

    # 훈련용 데이터, 검증용 데이터 설정
    X_train , y_train = X[train_idx] , y[train_idx] # 훈련용 데이터
    X_valid , y_valid = X[valid_idx] , y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train , y_train) # LightGBM 전용 훈련 데이터 셋
    dvalid = lgb.Dataset(X_valid , y_valid) # LightGBM 전용 검증 데이터 셋

    # LightGBM 모델 훈련
    lgb_model = lgb.train(params = params , # 훈련용 하이퍼파라미터
                          train_set = dtrain, # 훈련 데이터 셋
                          num_boost_round = 1000, # 부스팅 반복 횟수
                          valid_sets=  dvalid ,  # 성능 평가용 검증 데이터 셋
                          feval = gini, # 검증용 평가지표
                          early_stopping_rounds = 100, # 조기종료 조건
                          verbose_eval = 100 ) # 100번째마다 점수 출력

    # 테스트 데이터를 활용해 OOF 예측

    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits

    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측

    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    # 검증 데이터 예측 확률에 대한 정규화 지니계수

    gini_score = eval_gini(y_valid , oof_val_preds[valid_idx])
    print(f'폴드 {idx +1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.153356	valid_0's gini: 0.261373
[200]	valid_0's binary_logloss: 0.152424	valid_0's gini: 0.27589
[300]	valid_0's binary_logloss: 0.152031	valid_0's gini: 0.281949
[400]	valid_0's binary_logloss: 0.151802	valid_0's gini: 0.286194
[500]	valid_0's binary_logloss: 0.151737	valid_0's gini: 0.286881
[600]	valid_0's binary_logloss: 0.151686	valid_0's gini: 0.287701
[700]	valid_0's binary_logloss: 0.151677	valid_0's gini: 0.287901
Early stopping, best iteration is:
[655]	valid_0's binary_logloss: 0.151674	valid_0's gini: 0.287982
폴드 1 지니계수 : 0.2879824293420261

#########

In [22]:
print('OOF 검증 데이터 지니계수 : ' , eval_gini(y , oof_val_preds))

OOF 검증 데이터 지니계수 :  0.27992426994281666


In [23]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

## 성능 개선

In [24]:
all_data = pd.concat([train , test] , ignore_index = True)
all_data = all_data.drop('target' , axis =1 ) # 타깃값 제거

all_features = all_data.columns # 전체 피처

all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [25]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처

cat_features = [feature for feature in all_features if 'cat' in feature]

# 원-핫 인코딩 적용

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

## 파생 피처 추가

In [26]:
# 탐색적 데이터 분석과정에서는 필요 없는 피처를 추리는 것 외에 특별한 피처 엔지니어링 건을 찾아내지 못했다.

In [27]:
# 첫번째 , 한 데이터가 가진 결측값 개수를 파생 피처로 만들어본다. -1 이 결측값이므로 결측값 개수를 구하려면 -1 개수를 구하면 된다.

# 데이터 하나당 결측값 개수를 파생 피처로 추가

all_data['num_missing'] = (all_data == -1).sum(axis = 1)

In [28]:
# 명목형 피처 , calc 분류의 피처를 제외한 피처

remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature)]

# num_missing을 remaining_features 에 추가

remaining_features.append('num_missing')

remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'num_missing']

In [30]:
# 두번째, ind 분류의 피처들을 살펴본다. 모든 ind 피처 값을 연결해서 새로운 피처를 만들려고한다.

# 예를들어 , ps_ind_01 , ps_ind_02_car , ps_ind_03의 값이 각각 2, 3, 5 라면 모든 값을 연결해 2_2_5_로 만든다.

# ind 피처가 총 18개이므로 18개 값이 연결된다.

ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True

for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '-'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '-'

In [31]:
all_data['mix_ind']

0          2-2-5-1-0-0-1-0-0-0-0-0-0-0-11-0-1-0-
1           1-1-7-0-0-0-0-1-0-0-0-0-0-0-3-0-0-1-
2          5-4-9-1-0-0-0-1-0-0-0-0-0-0-12-1-0-0-
3           0-1-2-0-0-1-0-0-0-0-0-0-0-0-8-1-0-0-
4           0-2-0-1-0-1-0-0-0-0-0-0-0-0-9-1-0-0-
                           ...                  
1488023     0-1-6-0-0-0-1-0-0-0-0-0-0-0-2-0-0-1-
1488024    5-3-5-1-0-0-0-1-0-0-0-0-0-0-11-1-0-0-
1488025     0-1-5-0-0-1-0-0-0-0-0-0-0-0-5-0-0-1-
1488026    6-1-5-1-0-0-0-0-1-0-0-0-0-0-13-1-0-0-
1488027    7-1-4-1-0-0-0-0-1-0-0-0-0-0-12-1-0-0-
Name: mix_ind, Length: 1488028, dtype: object

In [32]:
# 세번째 , 명목형 피처의 고윳값별 개수를 새로운 피처로 추가한다. 고윳값별 개수는 value_counts()로 구한다.

# ps_ind_02_cat 피처의 고윳값별 개수 코드

all_data['ps_ind_02_cat'].value_counts()

 1    1079327
 2     309747
 3      70172
 4      28259
-1        523
Name: ps_ind_02_cat, dtype: int64